<a href="https://colab.research.google.com/github/kasidid-y/dormfood/blob/master/TextClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# reference : https://www.kaggle.com/code/karthik3890/text-clustering
# นายกษิดิศ อยู่คง 6530200576

#Data Collection
import pandas as pd
import re
data = pd.read_csv('testData500.csv')

In [3]:
#Data Preprocessing
partition = lambda x : 'negative' if x < 3 else 'positive'
actualScore = data['Score']
positiveNegative = actualScore.map(partition)
data['Score'] = positiveNegative

sorted_data = data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sno = nltk.stem.SnowballStemmer('english')
stop=set(stopwords.words('english'))
print(stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...


{"she's", "don't", 'what', 'by', 'then', 'should', 't', 'theirs', 'during', 'nor', 'above', 'on', 'weren', 'when', 're', "hasn't", 'or', 'am', 'hadn', 'they', 'is', 'same', 'this', 'wasn', 'll', 'hasn', 'so', 'through', 'will', 'isn', 'didn', "shan't", 'these', 'over', 'again', 'but', "you'll", 'you', 'are', "you're", 'most', 'mightn', 'both', 'don', "you've", "weren't", 'her', 'm', 'a', 'shouldn', 'which', 'themselves', 'just', 'ours', 'some', "mightn't", 'be', 'of', 'their', 'no', 'do', 'further', 'out', 'own', 'mustn', "wasn't", 'i', 'it', 'how', 'myself', 'o', 'haven', 'before', 'ma', 'shan', "needn't", "that'll", "should've", 'up', 'now', 'were', 'yours', 'wouldn', 'with', 'more', 'into', 'about', 'if', 'd', 'off', "isn't", 'him', 'having', 'against', 've', 's', 'we', 'couldn', 'where', 'under', 'aren', 'yourself', 'each', 'other', "hadn't", 'doesn', 'y', 'why', "shouldn't", 'she', 'all', "won't", "haven't", 'me', 'had', 'those', 'down', 'does', 'because', 'have', 'too', 'and', 't

[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Data Preprocessing
def cleanHtml(sentence):
    clean = re.compile('<.>*?')
    cleantext = re.sub(clean,' ',sentence)
    return cleantext

def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]', r'', sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ', cleaned) #`r` - raw data
    return cleaned

all_positive_words = []
all_negatie_words = []
final_string = []
i = 0
str1 = ''

for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanHtml(sent)
    for w in sent.split():
        for clean_words in cleanPunc(w).split():
            if( (clean_words.isalpha()) & (len(clean_words)>2) ):
                if(clean_words.lower() not in stop):
                    s = (sno.stem(clean_words.lower())).encode('utf8')

                    filtered_sentence.append(s)
                    if(final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)

                    if(final['Score'].values)[i] == 'negative':
                        all_negatie_words.append(s)
                else:
                    continue
            else:
                continue



    str1 = b" ".join(filtered_sentence) #`b` - bytes (byte data)
    final_string.append(str1)
    i+=1

#Feature engineering
final['CleanedText']=final_string
final['CleanedText']=final['CleanedText'].str.decode("utf-8")


<ipython-input-5-98a47e35a1c1>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['CleanedText']=final_string
<ipython-input-5-98a47e35a1c1>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['CleanedText']=final['CleanedText'].str.decode("utf-8")


In [ ]:
final.head()

In [ ]:
#Model development  : Vertorizing text
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(final['CleanedText'].values)
tfidf.shape

(498, 2763)

In [ ]:
#Model development : clustering
from sklearn.cluster import KMeans
model_tf = KMeans(n_clusters = 10,random_state=99)
model_tf.fit(tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=10, random_state=99)

In [ ]:
labels_tf = model_tf.labels_
cluster_center_tf = model_tf.cluster_centers_
labels_tf

array([1, 9, 1, 0, 0, 2, 1, 3, 7, 3, 9, 9, 9, 1, 9, 9, 9, 5, 7, 0, 7, 7,
       7, 1, 8, 6, 6, 6, 7, 6, 8, 8, 6, 6, 8, 6, 6, 8, 6, 6, 6, 6, 8, 2,
       0, 8, 6, 0, 6, 8, 6, 2, 2, 6, 6, 6, 2, 8, 6, 6, 8, 6, 1, 6, 1, 6,
       6, 6, 8, 8, 6, 8, 6, 6, 6, 1, 8, 8, 6, 6, 6, 6, 8, 2, 6, 8, 8, 6,
       1, 6, 8, 6, 6, 6, 8, 4, 6, 2, 6, 8, 0, 7, 7, 0, 0, 2, 2, 1, 2, 2,
       7, 9, 7, 7, 3, 1, 4, 1, 1, 1, 1, 3, 1, 1, 1, 7, 4, 4, 4, 2, 4, 4,
       4, 4, 4, 4, 7, 4, 4, 4, 4, 8, 7, 1, 1, 0, 2, 0, 1, 1, 1, 7, 7, 9,
       9, 1, 7, 9, 7, 7, 7, 9, 9, 0, 9, 9, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 7, 1, 8, 1, 2, 6, 9, 2, 7, 2, 7, 2, 7, 3, 1, 2, 1, 0, 1, 2,
       4, 1, 4, 2, 7, 4, 0, 4, 4, 4, 4, 4, 3, 7, 1, 2, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 8, 5, 0, 3, 3, 7, 2, 2, 0, 2, 7, 8, 7, 0,
       7, 7, 7, 7, 7, 2, 2, 0, 7, 8, 2, 1, 1, 1, 1, 1, 2, 1, 2, 7, 2, 7,
       3, 7, 1, 7, 0, 1, 0, 7, 4, 4, 1, 1, 1, 8, 1, 3, 7, 3, 3, 1, 3, 3,
       2, 2, 7, 7, 3, 5, 3, 3, 3, 3, 3, 3, 6, 1, 0,

In [ ]:
cluster_center_tf


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00375627,
        0.        ],
       [0.        , 0.00729866, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00086748, 0.00549877, 0.00093019, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00470011,
        0.        ],
       [0.        , 0.        , 0.00295798, ..., 0.        , 0.        ,
        0.00809952]])

In [ ]:
term = tfidf_vect.get_feature_names_out()

In [ ]:
term[10:100]
term.shape

(2763,)

In [ ]:
#Model evaluation
from sklearn import metrics
silhouette_score_tf = metrics.silhouette_score(tfidf, labels_tf, metric='euclidean')

silhouette_score_tf

0.012393010418607377

In [ ]:
df = final
df['Tfidf Clus Label'] = model_tf.labels_
df.head()

<ipython-input-13-b74473644fdc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tfidf Clus Label'] = model_tf.labels_


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText,Tfidf Clus Label
10,11,B0001PB9FE,A3HDKO7OW0QNK4,Canadian Fan,1,1,positive,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...,dont know cactus tequila uniqu combin ingredi ...,1
252,253,B0002567IW,A1SSKFPX72MSMR,"Janna M. Sicard ""missjanna""",0,0,negative,1344556800,Sad outcome,"Five minutes in, one tentacle was bitten off, ...",five minut one tentacl bitten ball insid crack...,9
187,188,B00029XIZI,A2S72TUJDQUBMH,Nadia,0,0,positive,1236124800,Miracle,My Scotties were full of hot spots and when I ...,scotti full hot spot use within week hot spot ...,1
412,413,B00061KYVI,A3JE18D4XD2PYF,pattymelt,0,0,positive,1282521600,best roast ever,This is the best way to cook a roast. Our loc...,best way cook roast local groceri went buis co...,0
411,412,B00061KYVI,A28RMYGO9RGA0K,marilyn,0,0,positive,1328140800,french's roast'n bags,i love this product cant find it locally. Some...,love product cant find local sometim littl way...,0


In [ ]:
df.groupby(['Tfidf Clus Label'])['Text'].count()

Tfidf Clus Label
0    36
1    85
2    46
3    50
4    30
5    26
6    43
7    98
8    49
9    35
Name: Text, dtype: int64

In [ ]:
print("Top terms per cluster:")
order_centroids = model_tf.cluster_centers_.argsort()[:, ::-1]
for i in range(10):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % term[ind], end='')
        print()

Top terms per cluster:
Cluster 0: store
 groceri
 local
 fast
 deliveri
 price
 product
 find
 deal
 good
Cluster 1: tast
 good
 order
 flavor
 hot
 love
 great
 got
 brand
 ive
Cluster 2: bag
 product
 great
 save
 get
 amazon
 love
 price
 eat
 use
Cluster 3: coffe
 energi
 drink
 one
 shot
 tast
 tri
 blend
 bean
 cup
Cluster 4: sugar
 better
 oatmeal
 raw
 instant
 regular
 much
 use
 brown
 mccann
Cluster 5: tea
 drink
 ahmad
 one
 tast
 tri
 year
 herbal
 sweet
 qualiti
Cluster 6: chip
 flavor
 bag
 kettl
 potato
 spici
 like
 love
 brand
 eat
Cluster 7: use
 like
 product
 love
 mint
 great
 ship
 candi
 price
 one
Cluster 8: chip
 salt
 flavor
 vinegar
 buy
 favorit
 best
 tri
 kettl
 good
Cluster 9: food
 dog
 cat
 eat
 allergi
 balanc
 feed
 dri
 natur
 one


In [ ]:
for i in range(10):
    print("3 review of assigned to cluster ", i)
    print("-" * 70)
    print(df.iloc[df.groupby(['Tfidf Clus Label']).groups[i][5]]['Text'])
    print('\n')
    print(df.iloc[df.groupby(['Tfidf Clus Label']).groups[i][10]]['Text'])
    print('\n')
    print(df.iloc[df.groupby(['Tfidf Clus Label']).groups[i][15]]['Text'])
    print('\n')
    print("_" * 70)


4 review of assigned to cluster  0
----------------------------------------------------------------------
Buyer Beware Please! This sweetener is not for everybody.  Maltitol is an alcohol sugar and can be undigestible in the body.  You will know a short time after consuming it if you are one of the unsuspecting many who cannot digest it by the extreme intestinal bloating and cramping and massive amounts of gas a person can experience. Nausea, diarrhea & headaches can also be experienced. I learned my lesson the hard way years ago when I fell in love with the sugar-free chocolates suzanne sommers used to sell.  I thought I'd found sugar-free chocolate nirvana at first taste but the bliss was short lived when the terrible side effects of maltitol kicked in.  The discomfort was unlike anything I had ever felt before.  I blew up like a balloon and had very painful abdominal cramping.  As the symptoms passed, this too was very unpleasant.  Though hard for me to believe a low calorie sweeten

In [ ]:
%%shell
jupyter nbconvert --to html /content/TextClusteringNoOutput.ipynb

In [ ]:
print(nltk.data.find('corpora/stopwords/english'))

/root/nltk_data/corpora/stopwords/english


In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()


In [ ]:
term

2763